## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Muskegon,US,43.3000,-86.1334,27.00,68,90,17.27,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,21.99,73,90,17.27,light snow
2,2,Faya,SA,18.3851,42.4509,55.40,82,0,5.75,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,59.00,41,40,4.61,scattered clouds
4,4,Hermanus,ZA,-34.4187,19.2345,66.99,77,2,4.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?\t"))
max_temp = float(input("What is the maximum temperature you would like for your trip?\t"))

What is the minimum temperature you would like for your trip?	75
What is the maximum temperature you would like for your trip?	90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,West Bay,KY,19.3667,-81.4167,79.00,78,75,14.97,broken clouds
8,8,Puerto Colombia,CO,10.9878,-74.9547,86.00,66,20,14.97,few clouds
9,9,Valdivia,CL,-39.8142,-73.2459,89.60,33,0,9.22,clear sky
13,13,Camocim,BR,-2.9022,-40.8411,80.53,79,100,10.71,moderate rain
15,15,Dolores,AR,-36.3132,-57.6792,89.62,29,0,16.82,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,West Bay,KY,19.3667,-81.4167,79.00,78,75,14.97,broken clouds
8,8,Puerto Colombia,CO,10.9878,-74.9547,86.00,66,20,14.97,few clouds
9,9,Valdivia,CL,-39.8142,-73.2459,89.60,33,0,9.22,clear sky
13,13,Camocim,BR,-2.9022,-40.8411,80.53,79,100,10.71,moderate rain
15,15,Dolores,AR,-36.3132,-57.6792,89.62,29,0,16.82,clear sky
...,...,...,...,...,...,...,...,...,...,...
657,657,Diamantino,BR,-14.4086,-56.4461,83.10,75,84,5.10,moderate rain
658,658,Acari,PE,-15.4311,-74.6158,82.11,47,54,10.69,broken clouds
659,659,Nioro,GM,13.3500,-15.7500,83.77,13,36,7.09,scattered clouds
660,660,Bubaque,GW,11.2833,-15.8333,78.53,73,80,12.44,broken clouds


In [7]:
# Recheck that there are no null values using count - all data should be same count
preferred_cities_df.count()

City_ID                192
City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,West Bay,KY,79.00,broken clouds,19.3667,-81.4167,
8,Puerto Colombia,CO,86.00,few clouds,10.9878,-74.9547,
9,Valdivia,CL,89.60,clear sky,-39.8142,-73.2459,
13,Camocim,BR,80.53,moderate rain,-2.9022,-40.8411,
15,Dolores,AR,89.62,clear sky,-36.3132,-57.6792,
...,...,...,...,...,...,...,...
657,Diamantino,BR,83.10,moderate rain,-14.4086,-56.4461,
658,Acari,PE,82.11,broken clouds,-15.4311,-74.6158,
659,Nioro,GM,83.77,scattered clouds,13.3500,-15.7500,
660,Bubaque,GW,78.53,broken clouds,11.2833,-15.8333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print(f"Hotel not found in {city}... skipping.")
        

Hotel not found in Sebina... skipping.
Hotel not found in Beboto... skipping.
Hotel not found in Edd... skipping.
Hotel not found in Khudumelapye... skipping.
Hotel not found in Goias... skipping.
Hotel not found in Abapo... skipping.
Hotel not found in Gamba... skipping.
Hotel not found in Lagos... skipping.
Hotel not found in Saint-Louis... skipping.
Hotel not found in Mayor Pablo Lagerenza... skipping.
Hotel not found in Thinadhoo... skipping.
Hotel not found in Tambopata... skipping.
Hotel not found in Nioro... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.


clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)
clean_hotel_df
    

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,West Bay,KY,79.00,broken clouds,19.3667,-81.4167,Calypso Cove
8,Puerto Colombia,CO,86.00,few clouds,10.9878,-74.9547,Hotel El Emigrante
9,Valdivia,CL,89.60,clear sky,-39.8142,-73.2459,Hostal Entre Rios
13,Camocim,BR,80.53,moderate rain,-2.9022,-40.8411,Hotel Ilha do Amor
15,Dolores,AR,89.62,clear sky,-36.3132,-57.6792,Apart Hotel - Arianos
...,...,...,...,...,...,...,...
646,Ilhabela,BR,79.57,scattered clouds,-23.7781,-45.3581,Ilha Flat Hotel
657,Diamantino,BR,83.10,moderate rain,-14.4086,-56.4461,Hotel Delta
658,Acari,PE,82.11,broken clouds,-15.4311,-74.6158,HOSPEDAJE XIMENA
660,Bubaque,GW,78.53,broken clouds,11.2833,-15.8333,"Casa Dora, Bubaque"


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))